In [1]:
import geopandas as gpd
import pandas as pd

from pathlib import Path

# Standard imports
import json
import yaml

import requests
from tqdm import tqdm

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Generate Isochrones around Health Facilities

This notebook requests isochrone contours around certain health facility points sourced from Geoportal. Isochrones are generated separately for each type of health facility. To generate the isochrones, we made use of Mapbox's API.

### Input
- Health Facility Points (health facility types: hospital, barangay health centers, regional health units)

### Output
- Each contour for each type saved to a GeoJSON

## Set-up input parameters and directories

In [2]:
HOSPITALS_FPATH = Path("../../../data/02-raw/philippines_healthfacilities.gpkg")

PROCSSED_DIR = Path("../../../data/03-processed/")
OUTPUT_DIR = Path("../../../data/04-output/")
ISOCHRONES_DIR = PROCSSED_DIR / "isochrones"

## Define Mapbox API functions

In [3]:
def get_mapbox_iso(df, profile, mins):
    """
    Gets isochrone via mapbox API.

    df: dataframe with separate columns for lat and lng.
    profile: mobility profiles, check API docs for more information.
    mins: number of minutes

    """
    id_processed = []
    _out = []
    _no_data = []

    with tqdm(total=df.shape[0]) as pbar:
        for uid, lat, lng in df.values:

            if not uid in id_processed:
                # mapbox URL
                iso_url = f"https://api.mapbox.com/isochrone/v1/mapbox/{profile}/{lng}%2C{lat}?contours_minutes={mins}&polygons=true&denoise=1&access_token={api_key}"
                r = requests.get(iso_url)

                if r.status_code == 200:

                    poly_gdf = gpd.read_file(json.dumps(r.json()), driver="GeoJSON")
                    poly_gdf["uid"] = uid

                    _out.append(poly_gdf)

                else:
                    print(f"{r.status_code} was returned for uid {uid}.")
                    _no_data.append(uid)

            pbar.update(1)

    # return gdf of contours
    if len(_out) != 0:
        out = pd.concat(_out)
        out.drop(
            columns=[
                "fill",
                "fillOpacity",
                "fill-opacity",
                "fillColor",
                "color",
                "opacity",
                "metric",
            ],
            inplace=True,
        )

    # return list of pois with no contour
    if len(_no_data) != 0:
        no_data = pd.concat(_no_data)
    else:
        no_data = []

    return out, no_data


def get_multi_iso(iso_minutes, health_gdf, type="hospital"):
    """
    Run Mapbox API and export contour to geojson.

    health_gdf: geodataframe for one health facility type
    type: health facility type

    """
    facility_iso_df, no_data = get_mapbox_iso(
        health_gdf[["id", "lat", "lng"]], "driving-traffic", iso_minutes
    )
    facility_iso_df.to_file(
        ISOCHRONES_DIR / f"iso_{type}_drivetraffic_{iso_minutes}.geojson",
        driver="GeoJSON",
    )

## Load hospital points

In [4]:
doh_gdf = gpd.read_file(HOSPITALS_FPATH, driver="GPKG")
doh_gdf.head(2)

,id,facilityco,healthfaci,typeofheal,barangay,municipali,province,region,status,address,style,geometry
0,1.0,DOH000000000002277,Calvario Barangay Health Station,Barangay Health Station,Calvario,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.98987 6.65182)
1,2.0,DOH000000000010319,Cabunbata Barangay Health Station,Barangay Health Station,Cabunbata,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.96630 6.67152)


In [5]:
doh_gdf.shape

(23676, 12)

In [6]:
doh_gdf = doh_gdf.to_crs("epsg:4326")

In [7]:
doh_gdf["lng"] = doh_gdf["geometry"].x
doh_gdf["lat"] = doh_gdf["geometry"].y

## Filter to target cities

In [8]:
target_cities = [
    "Navotas",
    "Mandaluyong",
    "Muntinlupa",
    "Dagupan City",
    "Palayan City",
    "Legazpi City",
    "Iloilo City",
    "Mandaue City",
    "Tacloban City",
    "Zamboanga City",
    "Cagayan de Oro City",
    "Davao City",
]
filtered_doh = doh_gdf[
    doh_gdf["municipali"].str.contains("|".join(target_cities), case=False)
]

In [9]:
# check if all cities are covered
filtered_doh["municipali"].unique()

array(['Zamboanga City', 'Cagayan De Oro City (Capital)', 'MANDAUE CITY',
       'ILOILO CITY (CAPITAL)', 'ILOILO CITY (CAPITAL)*', 'DAGUPAN CITY',
       'MANDALUYONG', 'NAVOTAS', 'MUNTINLUPA CITY', 'PALAYAN CITY',
       'DAVAO CITY', 'TACLOBAN CITY (CAPITAL)', 'LEGAZPI CITY (CAPITAL)'],
      dtype=object)

In [11]:
filtered_doh["typeofheal"].value_counts()

Barangay Health Station                           417
Rural Health Unit                                 119
Hospital                                          110
Birthing Home/Lying-in Clinic                      68
Medical Clinic                                      5
Infirmary                                           4
Social Hygiene Clinic                               3
Drug Abuse Treatment and Rehabilitation Center      1
Name: typeofheal, dtype: int64

In [12]:
# filter to hospitals
hospitals_gdf = filtered_doh[filtered_doh["typeofheal"] == "Hospital"]

# filter to barangay health centers
health_centers_gdf = filtered_doh[
    filtered_doh["typeofheal"] == "Barangay Health Station"
]

# filter to Rural Health Units (RHU)
rhu_gdf = filtered_doh[filtered_doh["typeofheal"] == "Rural Health Unit"]

## Generate Isochrones

### Get Mapbox API Key

Generate and use your own token from your Mapbox account. The `secrets/mapbox.yaml` file should NOT be commited to the repo. <br>
Instead, create your own file by following this format:

`api-key: <YOUR API KEY HERE>` <br>
`mapbox-style-xyz-url: <YOUR XYZ URL HERE>`

In [20]:
SECRETS_YAML = Path("../../../secrets/mapbox.yaml")
with open(SECRETS_YAML, "r") as f:
    secrets = yaml.safe_load(f)
    api_key = secrets["api-key"]

### Get 1min. intervals of isochrones

Take note that the API has a request limit of 300 requests per minute. Number of points to generate isochrones on directly translate to the number of requests. Each request can take a max of 4 contours.

In [23]:
# generate isochrones around hospitals
get_multi_iso("28, 29", health_gdf=hospitals_gdf)

100%|██████████| 110/110 [00:24<00:00,  4.56it/s]
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [23]:
# generate isochrones around the barangay health centers
get_multi_iso("29, 30", health_gdf=health_centers_gdf, type="brgy_healthcenters")

  0%|          | 0/417 [00:00<?, ?it/s]

100%|██████████| 417/417 [01:20<00:00,  5.15it/s]
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [30]:
# generate isochrones around the regional health units
get_multi_iso("29, 30", health_gdf=rhu_gdf, type="rhu")

100%|██████████| 119/119 [00:30<00:00,  3.92it/s]
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
